# Statistical Inference

In the slides you saw how, if you understand the sampling distribution of a statistic, you can compute a confidence interval. In this notebook, you'll see how to use Python to create these confidence intervals.

## Confidence Intervals

**Big Idea:** Create an interval so that C\% of the time, the parameter of interest lands within that interval (C is the confidence level - typically 95\% or 99\%).

A confidence interval lets you quantify the uncertainty associated with a point estimate. Wider interval means more uncertainty.

**General Formula for Confidence Interval:**

$$ \text{point estimate} \pm \text{margin of error} $$

In the case that you can approxmiate the sampling distribution of the statistic of interest (most commonly in the case of a mean or proportion), then you can express the margin of error in terms of the standard deviation of this distribution (the *standard error of the estimate*) and a critical value, which is based on the confidence level.

$$ \text{point estimate} \pm \text{standard error of estimate} \cdot \text{critical value}$$

Higher confidence means larger critical value and hence, a wider interval.

Also, you know that the standard error of the estimate shrinks as sample size increases, so you can get a narrower confidence interval by taking a larger sample.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
%matplotlib inline

## Confidence Intervals for the Mean

You have seen from the slides how to create a confidence interval for the mean. In Python, you'll use the `t.interval` function from `scipy.stats`.


In this example, you will be using data from the [American Time Use Survey](https://www.bls.gov/tus/). Specifically, you'll be looking at a sample of 50 adults and the number of minutes they reported sleeping for the survey day.

In [ ]:
sleeping = pd.read_csv('../data/atus_sleeping.csv')

In [ ]:
sleeping.head()

First, look at the distribution of the sample values.

In [ ]:
sleeping['minutes_spent_sleeping'].hist();

In order for your confidence interval to be valid, you need to make sure that the data is reasonably close to normally distributed. This can be relaxed if you have a large enough sample, but with only 50 observations, it is best to check for normality.

Recall that you can check for normality by looking at a Q-Q plot.

In [ ]:
from nssstats.plots import qq_plot

In [ ]:
qq_plot(sleeping['minutes_spent_sleeping'])

It is not perfect, but with the exception of a couple of very high values, it looks reasonably close to a normal distribution.

Now, find the point estimate you will use for the mean. You can use the sample mean as the point estimate:

In [ ]:
sleeping['minutes_spent_sleeping'].mean()

Also, look to see how much variability there is in the sample by calculating the standard deviation:

In [ ]:
sleeping['minutes_spent_sleeping'].std()

You can see that on average, people in this sample slept for 550.96 minutes (9.18 hours), but there is a lot a variability in the reported values. 

Armed with this knowledge, construct your confidence interval. Do this by using the `t` function from the scipy stats module.

In [ ]:
from scipy.stats import t, norm, sem

To use this function, you need to specify 4 things:
* alpha: The confidence level
* df: The number of degrees of freedom for the $t$-distribution, equal to the the sample size - 1
* loc: The sample mean
* scale: The standard error, equal to $\frac{s}{\sqrt{n}}$  

Let's first look at the standard error.

Recall that this is an estimate of the variability in the sample means across all possible samples with the same size as our sample.

You can manually calculate it: 

In [ ]:
sleeping['minutes_spent_sleeping'].std() / np.sqrt(len(sleeping['minutes_spent_sleeping']))

Or we can use the `sem` function:

In [ ]:
sem(sleeping['minutes_spent_sleeping'])

Passing all of these values into the `t.interval` function, you can get a 95% confidence interval as follows:

In [ ]:
t.interval(alpha = 0.95, 
           df = sleeping['minutes_spent_sleeping'].count() - 1, 
           loc = np.mean(sleeping['minutes_spent_sleeping']), 
           scale = sem(sleeping['minutes_spent_sleeping']))

You can be 95% confident that the population mean is between 507 minutes and 594 minutes.

In this case, what is the margin of error?  

For the case of a t-interval, the margin of error is equal to the distance from the point estimate to either the upper or lower bounds of the confidence interval. Recall from above that the point estimate was 550.96.

In [ ]:
550.96 - 507.88 

In [ ]:
594.04 - 550.96

Notice that, due to the symmetry of the confidence interval, you get the same margin of error for either difference.

What happens if we change our confidence level? Create a new confidence interval, but this time do an 80\% confidence interval.

In [ ]:
#Your code here

How does the margin of error change from the margin of error for the 95\% confidence interval?

In [ ]:
# Your code here

You can see that the margin or error is smaller. However, you are less confident that the true population mean is in this interval.

Now, create a 99\% confidence interval.

In [ ]:
# Your code here

The margin of error now is much larger:

In [ ]:
# Your code here

This demonstrates the key tradeoff when building confidence intervals. If you want to be more confident that the confidence interval contains the true population parameter, you need to increase the margin of error.

Now, let's estimate the average number of crashes that occur in Fridays.

In [ ]:
crashes = pd.read_csv('../data/friday_crashes.csv')

In [ ]:
crashes.head()

Let's look at the distribution:

In [ ]:
crashes['Accident Number'].hist();

And look at a Q-Q plot to assess normality:

In [ ]:
qq_plot(crashes['Accident Number'])

First, get your point estimate:

In [ ]:
crashes['Accident Number'].mean()

**Exercise:** Fill in the code to construct a 95% confidence interval for the mean number of crashes on Friday and then find the margin of error.

In [ ]:
# Your code here

This makes the margin of error:

In [ ]:
# Your code here

## Confidence Intervals for the Proportion

For this example, you'll be working wit a sample of 250 participants in the 2015 [National Health and Nutrition Examination Survey](https://www.cdc.gov/nchs/nhanes/index.htm) who were at least 18 years old.

Specifically, using this sample, estimate the proportion of US adults who smoke. In this dataset, there is a column indicating whether the person smoked tobacco in the last five days, so you can use this to make your estimate.

In [ ]:
smoking = pd.read_csv('../data/NHANES_smoking.csv')
smoking.head()

First, let's take a look at the data and see how it is divided between the two categories:

In [ ]:
smoking['smoked_in_last_5_days'].value_counts()

In [ ]:
smoking['smoked_in_last_5_days'].value_counts(normalize = True)

Based on the sample, the point estimate for the proportion of adults who smoked in the last 5 days is 21.6%.

Now, create our confidence interval.

Confidence intervals for the proportion can be constructed using the `proportion_confint` function from the `statsmodels.stats.proportion` module.

In [ ]:
from statsmodels.stats.proportion import proportion_confint

To use this function, you need to specify
* count: The number of "successes"
* nobs: the total number of observations
* alpha: 1 - confidence level

**Warning:** This function treats alpha differently than the confidence interval function above, where alpha corresponded to the confidence level.

If you want to get the count of people who did smoke in the last 5 days, you can use `.loc`

In [ ]:
smoking['smoked_in_last_5_days'].value_counts().loc[True]

In [ ]:
proportion_confint( count = smoking['smoked_in_last_5_days'].value_counts().loc[True],
                   nobs = smoking['smoked_in_last_5_days'].count(),
                   alpha = 0.05
                  )

This shows that we can be 95% confident that the true proportion of US adults who regularly smoke tobacco is between 16.5% and 26.7%.